In [1]:
cd ../..

e:\GitHub\computervision


In [2]:
import torch
import os

import numpy as np
import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torchvision import models, transforms
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING = False
MODEL_WEIGHTS_PATH = 'notebooks/efficientnet/models//'
MODEL_LAST_EPOCH = '.pth'

# CONFIG VARIABLES
DATA_DIR = 'E:/Datasets/cor-splits/sgkf-8-1-1-2000'
START_EPOCH = 1 # Change it if you're continuing an existing run
N_EPOCHS = 50
## Learning Rate Scheduler
LR = 0.001  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 10  # Number of epochs before altering LR
LR_GAMMA = 0.1
SAVE_INTERVAL = 5 # How many epochs between each backup
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0

In [4]:
working_folder = 'notebooks/vit'
models_folder = 'models'
from datetime import datetime

curr_time = datetime.now()
# curr_time = datetime(curr_time.year, curr_time.month, curr_time.day, curr_time.hour)
curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

output_folder = os.path.join(working_folder, models_folder)
output_folder = os.path.join(output_folder, curr_time)

In [5]:
# vit_b_16
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256, transforms.InterpolationMode('bilinear')),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [6]:
model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)

for param in model.parameters(): # Turning parameters not trainable
    param.requires_grad = True

# A newly defined layer is created with requires_grad=True by default
model.heads = nn.Sequential(
    nn.Linear(in_features=768, out_features=2)
    )

if CONTINUE_TRAINING:
    output_folder = MODEL_WEIGHTS_PATH
    model.load_state_dict(torch.load(output_folder+MODEL_LAST_EPOCH))
else:
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
       
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# optimizer = optim.SGD(model.heads.parameters(), lr=LR)
optimizer = optim.SGD(model.parameters(), lr=LR)
# optimizer = optim.Adam(model.parameters(), lr=LR)

# Learning rate scheduling should be applied after optimizer’s update
# e.g., you should write your code this way:
# for epoch in range(100):
#     train(...)
#     validate(...)
#     scheduler.step()

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA)

In [7]:
dataset = PytorchDataset(DATA_DIR, data_transforms)
trainer = PytorchTraining(device, dataset, output_folder)

model = trainer.train_pytorch_model(model, criterion, optimizer, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL)

Epoch 1/50
----------
train Loss: 0.6797 Acc: 0.5750
val Loss: 0.6008 Acc: 0.7042
Epoch duration: 6 m 8s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 2/50
----------
train Loss: 0.5372 Acc: 0.7801
val Loss: 0.5275 Acc: 0.7723
Epoch duration: 11 m 21s
Learning Rate = 0.001
Lower loss found, resetting patience counter

Epoch 3/50
----------
train Loss: 0.4444 Acc: 0.8580
val Loss: 0.4763 Acc: 0.7746
Epoch duration: 7 m 57s
Learning Rate = 0.001
Loss didnt decrease. Increasing patience counter

Epoch 4/50
----------
train Loss: 0.3701 Acc: 0.8936
val Loss: 0.4456 Acc: 0.7840
Epoch duration: 11 m 17s
Learning Rate = 0.001
Loss didnt decrease. Increasing patience counter

Epoch 5/50
----------
train Loss: 0.3081 Acc: 0.9209
val Loss: 0.4215 Acc: 0.7840
Epoch duration: 11 m 6s
Learning Rate = 0.001
Loss didnt decrease. Increasing patience counter

Epoch 6/50
----------


KeyboardInterrupt: 